In [1]:
import pandas as pd

dataset = pd.read_csv('BOW_dataset_v1.csv', encoding='utf-8')
# dataset = dataset.
# dataset = dataset.loc[dataset.ML_class == 'series_and_sequence']
dataset

,Unnamed: 0,(binomial),(d),(d)+(d)+(d)+,(d)cm,(d)m,(d)n,(d)nd,(d)rd,(d)s_(d),...,would,write,x,yaya,year,yearly,years,zero,zeros,ML_class
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,series_and_sequence
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,series_and_sequence
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,series_and_sequence
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,series_and_sequence
4,4,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,series_and_sequence
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
949,949,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,polynomial_equation
950,950,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,polynomial_equation
951,951,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,polynomial_equation
952,952,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,polynomial_equation


In [2]:
from sklearn.metrics.pairwise import linear_kernel
cos_sim_data = dataset.drop(columns = ['ML_class', 'Unnamed: 0'])
cos_sim = linear_kernel(cos_sim_data, cos_sim_data)

In [3]:
questions = pd.read_csv('dataset_after_transformation.csv', encoding='utf-8')
question_bank = pd.read_csv('initial_dataset.csv', encoding='utf-8') 


In [4]:
indices = pd.Series(questions['Unnamed: 0'].to_list(), index = questions['question'])
pd.DataFrame(indices)

,0
question,
It is the sum of the terms of Symbol(x) sequence,0
Find the sum of the first ten terms of the arithmetic sequence Integer((d)),1
Find the sum of the first Integer((d)) terms of the arithmetic sequence Integer((d)) Integer((d)) Integer((d)) …,2
The sum of the first Integer((d)) terms of an arithmetic sequence is Integer((d)) What is the first term if the last term is Integer((d)) and the common difference is Integer((d)),3
The third term of an arithmetic sequence is −(d) and the seventh term is Integer((d)) What is the sum of the first Integer((d)) terms,4
...,...
Factor Add(Mul(Integer(-(d))Integer((d))Pow(Symbol(x)Integer((d))))Mul(Integer((d))Symbol(x))Mul(Integer(-(d))Integer((d)))),949
Factor Add(Mul(Integer(-(d))Integer((d))Pow(Integer((d))Integer(-(d)))Pow(Symbol(x)Integer((d))))Mul(Integer(-(d))Mul(Integer((d))Pow(Integer((d))Integer(-(d)))Pow(Symbol(x)Integer((d)))))Mul(Integer(-(d))Mul(Integer((d))Pow(Integer((d))Integer(-(d)))Pow(Symbol(x)Integer((d)))))),950
What is Symbol(x) in Add(Mul(Integer((d))Pow(Symbol(x)Integer((d))))Mul(Integer(-(d))Mul(Integer((d))Pow(Symbol(x)Integer((d)))))Mul(Integer((d))Symbol(x))Mul(Integer(-(d))Integer((d)))) Integer((d)),951


In [7]:
from latex2sympy2 import latex2sympy
from sympy import *
from sympy.parsing.sympy_parser import parse_expr
import re


#Changes Before Sympy
def change_to_dollar(exp):
    exp = re.sub("\.\.\.|\\dots|…","x", exp)
    exp = re.sub("=","$ $", exp)
    return exp

def expression_checker(question):
    qs = question
    qs = re.sub(r"\\sqrt\{(.+?)\}", r"(\1)^{\\frac{1}{2}}", qs)
    qs = re.sub(r"[\\][(][)][\\]", "$", qs)
    fx = re.search('[$].+?[=].+?[$]',qs)
    len_check = len(re.findall('[$].+?[=].+?[$]', qs))
    iplus = 0
    while iplus<len_check:
        iplus += 1
        qs = qs[:fx.span()[0]] + " " + change_to_dollar(fx.group()) + " " + qs[fx.span()[1]:]
        #check for the next LaTex Enclosure if any         
        fx = re.search('[$].+?[=].+?[$]', qs)
    #the new question is added to index i       
    return qs

def sympify(question):
    test = question
    #regex value for searching all characters from '$' to the next '$' symbol
    check = re.search('[$].+?[$]',test)
    #checks how many instance of latex enclosure are there in the question     
    check_len = len(re.findall('[$].+?[$]',test))
    iteration = 0
    while iteration<check_len:
        iteration += 1
        print(check.group())
        test = test[:check.span()[0]] + " " + re.sub(" ","",str(latex2sympy(r"{}".format(check.group())))) + " " + test[check.span()[1]:]   
        check = re.search('[$].+?[$]',test)       
    return test

# Change Must Happen After Sympy 
def change_characters(question):   
    question = re.sub(' \+ ', '+',question)
    question = re.sub(' - ', '-',question)
    question = re.sub(' \+', '+',question)
    question = re.sub(' -', '-',question)
    question = re.sub('\+ ', '+',question)
    question = re.sub('- ', '-',question)
    question = re.sub('=', ' ',question)
    question = re.sub('\n','',question)
    question = re.sub('\?','',question)
    question = re.sub('\'','',question)
    question = re.sub(',','',question)
    question = re.sub('[\.]','',question)
    question = re.sub('\t', ' ',question)
    question = re.sub(r"\_\_+", "__",question)
    question = re.sub('[;]', ' ',question)
    question = re.sub(':', ' ',question)
    return question
    

def tokenize_formula(expression):
    try:
        return srepr(parse_expr(expression, evaluate=False))
    except:
        return expression

def expression_treefy(question):
    item_question = question
    tokenize_question_item = item_question.split()
    for x in range(len(tokenize_question_item)):
        check = re.findall('[\+]|[\*]|[-]|[/]|[0-9]',tokenize_question_item[x])
        if len(tokenize_question_item[x]) == 1:
            check = True 
        if check: 
            print(tokenize_question_item[x])
            text = re.sub("\s|[,]","",str(tokenize_formula(tokenize_question_item[x])))
            print(text)
            text = re.sub("\d+","(d)",text)
            text = re.sub("[(]['].+?['][)]","(x)",text)
#             text = re.sub("Symbol[(]['].+?['][)]","symbol(x)",text)
            text = re.sub('\s+', '', text)
            tokenize_question_item[x] = text
    item_question = ' '.join(tokenize_question_item)
    print(item_question)
    return item_question

def convert(cleaned_question):
    question = cleaned_question
    question = expression_checker(question)
#     print("Step 1: Removing Equal Sign Inside LaTex So That Sympy would not evaluate the equation or answer the equation")
#     print(question)
    question = sympify(question)
#     print("Step 2: Converting LaTex Expressions into Freeform Structure")
#     print(question)
    question = change_characters(question)
#     print("Step 3: Removing Unnecessary Characters")
#     print(question)
    question = expression_treefy(question)
#     print("Step 4: Expressing Freeform Equations into Expression trees")
#     print(question)
    return question

def get_recommendations (qs, cos_sim = cos_sim):
    idx = indices[qs]
    print(idx)
    sim_scores = enumerate(cos_sim[idx])
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse = True)
    sim_scores = sim_scores[:20]
    sim_index = [i[0] for i in sim_scores]
    print(sim_scores)
    dataframe = question_bank.iloc[sim_index]
    return dataframe

tokenized = convert(r'Find the sum of the first ten terms of the arithmetic sequence 4,10,16')
print(tokenized)
get_recommendations(tokenized)

41016
Integer(41016)
Find the sum of the first ten terms of the arithmetic sequence Integer((d))
Find the sum of the first ten terms of the arithmetic sequence Integer((d))
1
[(18, 12.0), (32, 11.0), (34, 11.0), (45, 11.0), (2, 10.0), (3, 10.0), (5, 10.0), (19, 10.0), (33, 10.0), (44, 10.0), (46, 10.0), (48, 10.0), (7, 9.0), (8, 9.0), (16, 9.0), (21, 9.0), (39, 9.0), (40, 9.0), (50, 9.0), (51, 9.0)]


,Unnamed: 0,question,label
18,18,Find the sum of the first 16 terms of the arit...,series_and_sequence
32,32,Find the sum of the first 15 terms of the arit...,series_and_sequence
34,34,Find the sum of the first 19 terms of the arit...,series_and_sequence
45,45,Find the sum of the first 15 terms of the arit...,series_and_sequence
2,2,Find the sum of the first 25 terms of the arit...,series_and_sequence
3,3,The sum of the first 10 terms of an arithmetic...,series_and_sequence
5,5,Find the sum of the first 50 terms of the arit...,series_and_sequence
19,19,"If the first n terms of the sequence: 9, 12, 1...",series_and_sequence
33,33,Find the sum of the first 11 terms of the arit...,series_and_sequence
44,44,Find the sum of the first 12 terms of the arit...,series_and_sequence


IndexError: single positional indexer is out-of-bounds

In [18]:
dataset.tail()

,Unnamed: 0,(binomial),(d),(d)+(d)+(d)+,(d)cm,(d)m,(d)n,(d)nd,(d)rd,(d)s_(d),...,yaya,year,yearly,years,zero,zeros,ML_class,0,2,3
950,950.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,polynomial_equation,NaN,NaN,NaN
951,951.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,polynomial_equation,NaN,NaN,NaN
952,952.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,polynomial_equation,NaN,NaN,NaN
953,953.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,polynomial_equation,NaN,NaN,NaN
955,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
